# 数据处理

In [1]:
import pandas as pd

# 文件路径列表
file_names = [
    '/home/mw/input/dataset7967/Health_20_0.csv',
    '/home/mw/input/dataset7967/Chipped_20_0.csv',
    '/home/mw/input/dataset7967/Miss_20_0.csv',
    '/home/mw/input/dataset7967/Root_20_0.csv',
    '/home/mw/input/dataset7967/Surface_20_0.csv',
    '/home/mw/input/dataset7967/ball_20_0.csv',  # 这个文件特殊处理
    '/home/mw/input/dataset7967/inner_20_0.csv',
    '/home/mw/input/dataset7967/outer_20_0.csv'
]

# 列名列表
columns_name = [
    'Health_20_0', 'Chipped_20_0', 'Miss_20_0', 'Root_20_0',
    'Surface_20_0', 'ball_20_0', 'inner_20_0', 'outer_20_0'
]

# 初始化空的 DataFrame
data_1 = pd.DataFrame()

# 遍历文件列表
for index in range(len(file_names)):
    try:
        # 读取 CSV 文件，假设文件是以制表符分隔的
        data = pd.read_csv(file_names[index], sep="\t", header=None, low_memory=False)
        
        # 提取 16-103440 行的数据
        data1 = data.iloc[1015:202063]
        
        # 只保留前 8 列
        data2 = data1.iloc[:, :8]
        
        # 提取第三列数据
        data3 = data2.iloc[:, 2]
        
        # 将数据存储到 DataFrame 中
        data_1[columns_name[index]] = data3.reset_index(drop=True)[:201048]
    
    except Exception as e:
        # **如果是 ball_20_0.csv，则单独处理**
        if "ball_20_0.csv" in file_names[index]:
            try:
                data = pd.read_csv(file_names[index], header=None, low_memory=False)

                # 提取 16-103440 行的数据
                data1 = data.iloc[1015:202063]

                # 只保留前 8 列
                data2 = data1.iloc[:, :8]

                # 确保 data2 至少有 3 列
                if data2.shape[1] >= 3:
                    data3 = data2.iloc[:, 2]
                    data_1["ball_20_0"] = data3.reset_index(drop=True)[:201048]

            except Exception as e:
                print(f"Failed to process ball_20_0.csv manually: {e}")

# 保存结果到 CSV 文件
data_1.to_csv('data_1.csv', index=False)


## 数据标准化及分割

In [3]:
import numpy as np
import pandas as pd

data_array = data_1.values

# 窗口大小和步长
window_size = 2048
step_size = 1000

# 存储分割后的数据
segmented_data = []
labels = []

# 遍历每个类别的列，进行窗口滑动分割
for label, column_index in enumerate(range(data_array.shape[1])):
    column_data = data_array[:, column_index]  # 获取当前类别的数据
    num_samples = (len(column_data) - window_size) // step_size + 1  # 计算可分割的样本数

    for i in range(num_samples):
        start = i * step_size
        end = start + window_size
        segmented_data.append(column_data[start:end])  # 提取窗口数据
        labels.append(label)  # 记录对应的类别

# 转换为 NumPy 数组
segmented_data = np.array(segmented_data)
labels = np.array(labels)

# 将数据保存为 DataFrame
data_2 = pd.DataFrame(segmented_data)
data_2['labels'] = labels  # 添加标签列

# 保存为 CSV 文件
data_2.to_csv('data_2.csv', index=False)
data_2.shape

(1600, 2049)

# 浅层学习部分的数据

In [4]:
import pywt
import numpy as np
import pandas as pd
from scipy.stats import kurtosis, skew
from scipy.signal import welch
from numpy.linalg import svd

# --- 参数设定 ---
wavelet = 'db4' # 小波基函数
level = 4      # 分解层数


# --- 特征提取函数 ---
def extract_time_features(reconstructed_signal, cA, cD):
    """
    从重构信号计算时域统计特征，
    从原始分解系数计算能量特征。
    """
    reconstructed_signal = np.asarray(reconstructed_signal, dtype=np.float64)
    cA = np.asarray(cA, dtype=np.float64)
    cD = [np.asarray(d, dtype=np.float64) for d in cD]

    # 基于重构信号的特征
    max_val = np.max(reconstructed_signal)
    min_val = np.min(reconstructed_signal)
    mean_val = np.mean(reconstructed_signal)
    peak_val = np.max(np.abs(reconstructed_signal))
    peak_to_peak = max_val - min_val
    kurt = kurtosis(reconstructed_signal)
    skewness = skew(reconstructed_signal)
    rms = np.sqrt(np.mean(reconstructed_signal**2))
    rectified_mean = np.mean(np.abs(reconstructed_signal))
    waveform_factor = rms / rectified_mean if rectified_mean != 0 else 0
    peak_factor = peak_val / rms if rms != 0 else 0
    sqrt_abs_mean = np.mean(np.sqrt(np.abs(reconstructed_signal)))
    margin_factor = peak_val / (sqrt_abs_mean ** 2) if sqrt_abs_mean != 0 else 0

    # 基于原始分解系数的能量特征
    energy_high = sum(np.sum(d**2) for d in cD) # 高频细节系数能量
    energy_low = np.sum(cA**2)                 # 低频近似系数能量

    return [max_val, min_val, mean_val, peak_val, peak_to_peak, kurt, skewness, rms,
            rectified_mean, waveform_factor, peak_factor, margin_factor, energy_high, energy_low]


def extract_frequency_features(reconstructed_signal, fs=1.0):
    """从重构信号计算频域特征 (Welch法)。"""
    signal = np.asarray(reconstructed_signal, dtype=np.float64)
    # 使用min(len(signal), 256)确保nperseg不超过信号长度
    freqs, psd = welch(signal, fs=fs, nperseg=min(len(signal), 256))
    psd = psd.astype(np.float64)

    sum_psd = np.sum(psd)
    if sum_psd == 0:
        return [0, 0, 0, 0, 0] # 如果PSD全零，返回0

    centroid_freq = np.sum(freqs * psd) / sum_psd
    mean_square_freq = np.sum((freqs**2) * psd) / sum_psd
    freq_variance = mean_square_freq - centroid_freq**2
    freq_band_energy = sum_psd
    freq_std = np.sqrt(freq_variance) if freq_variance >= 0 else 0

    return [centroid_freq, mean_square_freq, freq_variance, freq_band_energy, freq_std]


def extract_svd_features(cA, cD):
    """从原始分解系数计算SVD特征。"""
    cA = np.asarray(cA, dtype=np.float64).ravel()
    cD_flat = [np.asarray(d, dtype=np.float64).ravel() for d in cD]
    high_freq_components = np.concatenate(cD_flat) if cD_flat else np.array([], dtype=np.float64)

    # 低频奇异值
    low_freq_svd = 0
    if cA.size > 0:
        U_a, S_a, V_a = svd(cA.reshape(-1, 1), full_matrices=False)
        if len(S_a) > 0:
            low_freq_svd = S_a[0]

    # 高频奇异值
    high_freq_svd = 0
    if high_freq_components.size > 0:
        U_d, S_d, V_d = svd(high_freq_components.reshape(-1, 1), full_matrices=False)
        if len(S_d) > 0:
            high_freq_svd = S_d[0]

    return [high_freq_svd, low_freq_svd]


# --- 主处理流程 ---
features_matrix = []

for sample in segmented_data:
    sample = np.asarray(sample, dtype=np.float64).ravel()

    # 1. 小波分解
    coeffs = pywt.wavedec(sample, wavelet, level=level)
    cA = coeffs[0]
    cD = coeffs[1:]

    # 2. 小波重构
    reconstructed_sample = pywt.waverec(coeffs, wavelet)
    # 可选: 如果需要，截断/填充以匹配原始长度
    # reconstructed_sample = reconstructed_sample[:len(sample)]

    # 3. 提取特征
    time_features = extract_time_features(reconstructed_sample, cA, cD) # 时域(重构信号)+能量(系数)
    freq_features = extract_frequency_features(reconstructed_sample)     # 频域(重构信号)
    svd_features = extract_svd_features(cA, cD)                         # SVD(系数)

    # 合并特征
    features_matrix.append(time_features + freq_features + svd_features)

# --- 创建并保存DataFrame ---
feature_columns = [
    "最大值", "最小值", "平均值", "峰值", "峰峰值", "峭度", "偏度", "均方根",
    "整流平均值", "波形因子", "峰值因子", "裕度因子", "高频能量", "低频能量",
    "重心频率", "均方频率", "频率方差", "频带能量", "频率标准差",
    "高频奇异值特征", "低频奇异值特征"
]

features_df = pd.DataFrame(features_matrix, columns=feature_columns)
features_df["故障类别"] = labels # 直接添加标签列，假设长度匹配

# 保存结果
features_df.to_csv(
    'features_reconstructed_simplified_df.csv',
    index=False,
    encoding='utf-8-sig',
    float_format='%.3f'  # <--- 添加此参数
)
print("特征已提取并保存到 features_reconstructed_simplified_df.csv")

特征已提取并保存到 features_reconstructed_simplified_df.csv


## 随机森林

In [5]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns # 用于绘图

# 提取特征和标签
X = features_df.iloc[:, :-1]  # 特征
y = features_df.iloc[:, -1]   # 标签
# 打乱数据
features_df = features_df.sample(frac=1, random_state=42).reset_index(drop=True)

# 训练随机森林进行特征重要性评估
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y)

# 获取特征重要性
feature_importances = rf.feature_importances_

# 创建特征重要性数据框
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
}).sort_values(by="Importance", ascending=False)
feature_importance_df


,Feature,Importance
8,整流平均值,0.137515
13,低频能量,0.106784
7,均方根,0.106127
20,低频奇异值特征,0.102625
17,频带能量,0.089164
12,高频能量,0.086777
19,高频奇异值特征,0.084217
2,平均值,0.054439
15,均方频率,0.039433
16,频率方差,0.031384


In [6]:
plt.figure(figsize=(10, 8))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df, palette='viridis')
plt.title("Random Forest Feature Importances")
plt.xlabel("Importance Score")
plt.ylabel("Features")
plt.show()

<Figure size 720x576 with 1 Axes>

## 改进HHO算法

In [8]:
import numpy as np

class HarrisHawkOptimization:
    def __init__(self, fitness_function, n_hawks, dim, max_iter, lb, ub):
        """
        Harris Hawk Optimization (HHO) Algorithm

        Parameters:
            fitness_function (callable): 要优化的目标函数，返回 (score, params)
            n_hawks (int): 鹰群数量
            dim (int): 参数维度
            max_iter (int): 最大迭代次数
            lb (list or np.ndarray): 每个参数的下界
            ub (list or np.ndarray): 每个参数的上界
        """
        self.fitness_function = fitness_function
        self.n_hawks = n_hawks
        self.dim = dim
        self.max_iter = max_iter
        self.lb = np.array(lb)
        self.ub = np.array(ub)
        self.stagnation_count = 0

    def optimize(self, verbose=True):
        hawks = np.random.uniform(self.lb, self.ub, (self.n_hawks, self.dim))
        fitness = np.zeros(self.n_hawks)
        params_list = [None] * self.n_hawks

        for i in range(self.n_hawks):
            fitness[i], params_list[i] = self.fitness_function(hawks[i])

        best_idx = np.argmin(fitness)
        rabbit_pos = hawks[best_idx].copy()
        rabbit_energy = fitness[best_idx]
        best_params = params_list[best_idx]

        global_best_pos = rabbit_pos.copy()
        global_best_energy = rabbit_energy
        global_best_params = best_params

        last_rabbit_energy = rabbit_energy

        for t in range(self.max_iter):
            E0 = 2 * np.random.random() - 1
            E = 2 * E0 * (1 - (t / self.max_iter) ** 2)

            stagnation_limit = int(2 + 3 * (t / self.max_iter))

            for i in range(self.n_hawks):
                if abs(E) >= 1:
                    if np.random.random() < 0.5:
                        r_idx = np.random.randint(self.n_hawks)
                        hawks[i] = hawks[r_idx] - np.random.random() * abs(
                            hawks[r_idx] - 2 * np.random.random() * hawks[i])
                    else:
                        hawks[i] = (rabbit_pos - hawks.mean(0)) * np.random.random() + self.lb + \
                                   np.random.random() * (self.ub - self.lb)
                else:
                    r = np.random.random()
                    J = 2 * (1 - np.random.random())
                    if r >= 0.5 and abs(E) >= 0.5:
                        hawks[i] = rabbit_pos - E * abs(rabbit_pos - hawks[i])
                    elif r >= 0.5 and abs(E) < 0.5:
                        hawks[i] = rabbit_pos - E * abs(2 * rabbit_pos - hawks[i])
                    elif r < 0.5 and abs(E) >= 0.5:
                        hawks[i] = rabbit_pos - E * abs(J * rabbit_pos - hawks[i])
                    else:
                        hawks[i] = rabbit_pos - E * abs(J * rabbit_pos - hawks[i])

                hawks[i] = np.clip(hawks[i], self.lb, self.ub)

                curr_fitness, curr_params = self.fitness_function(hawks[i])
                if curr_fitness < fitness[i]:
                    fitness[i] = curr_fitness
                    params_list[i] = curr_params
                if curr_fitness < rabbit_energy:
                    rabbit_energy = curr_fitness
                    rabbit_pos = hawks[i].copy()
                    best_params = curr_params

                if curr_fitness < global_best_energy:
                    global_best_energy = curr_fitness
                    global_best_pos = hawks[i].copy()
                    global_best_params = curr_params

            improvement = abs(last_rabbit_energy - rabbit_energy)
            if improvement < 1e-6:
                self.stagnation_count += 1
                if self.stagnation_count >= stagnation_limit:
                    perturbation_scale = 0.1 * (1 - t / self.max_iter)
                    rabbit_pos += np.random.uniform(-perturbation_scale, perturbation_scale, self.dim) * (
                        self.ub - self.lb)
                    rabbit_pos = np.clip(rabbit_pos, self.lb, self.ub)
                    rabbit_energy, best_params = self.fitness_function(rabbit_pos)

                    if rabbit_energy < global_best_energy:
                        global_best_energy = rabbit_energy
                        global_best_pos = rabbit_pos.copy()
                        global_best_params = best_params
                    self.stagnation_count = 0
            else:
                self.stagnation_count = 0

            last_rabbit_energy = rabbit_energy

            if verbose:
                print(f"\n迭代次数{t + 1}/{self.max_iter}")
                print(f"当前准确率: {-rabbit_energy:.4f}")
                print(f"全局最佳准确率: {-global_best_energy:.4f}")
                print(f"最优参数: {best_params}")

        return global_best_pos, -global_best_energy, global_best_params


## BP神经网络(建议到本地IDE去跑)

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from pylab import mpl
from hho import HarrisHawkOptimization

# 设置中文显示
mpl.rcParams['font.sans-serif'] = ['STZhongsong']
mpl.rcParams['axes.unicode_minus'] = False

# 读取数据
file_path = "features_reconstructed_simplified_df.csv"
df = pd.read_csv(file_path)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
selected_features = ["整流平均值", "均方根", "低频能量", "低频奇异值特征", "高频能量", "频带能量", "平均值", "均方频率", "重心频率"]
X = df[selected_features]
y = df["故障类别"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def fitness_function(params):
    hidden1, hidden2, learning_rate = int(params[0]), int(params[1]), params[2]
    hidden1 = max(10, min(hidden1, 200))  # 限制范围
    hidden2 = max(5, min(hidden2, 200))
    learning_rate = round(max(0.0001, min(learning_rate, 0.1)), 9)

    clf = MLPClassifier(
        hidden_layer_sizes=(hidden1, hidden2),
        activation='relu',
        solver='adam',
        learning_rate_init=learning_rate,
        max_iter=2000,
        random_state=42,
        early_stopping=True,
        validation_fraction=0.1
    )
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    return -acc, (hidden1, hidden2, learning_rate)  # 负数用于最小化目标

# 设置搜索边界
lb = [50, 50, 0.001]
ub = [200, 200, 0.1]

# 初始化 HHO
hho = HarrisHawkOptimization(
    fitness_function=fitness_function,
    n_hawks=20,
    dim=3,
    max_iter=100,
    lb=lb,
    ub=ub
)

# 执行优化
_, best_accuracy, best_params = hho.optimize()

hidden1, hidden2, learning_rate = best_params

final_model = MLPClassifier(
    hidden_layer_sizes=(hidden1, hidden2),
    activation='relu',
    solver='adam',
    learning_rate_init=learning_rate,
    max_iter=2000,
    random_state=42,
    early_stopping=True,
    validation_fraction=0.1
)

final_model.fit(X_train_scaled, y_train)
y_pred = final_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"\n=== 优化完成 ===")
print(f"最佳参数 - 隐藏层1: {hidden1}, 隐藏层2: {hidden2}, 学习率: {learning_rate}")
print(f"最终准确率: {accuracy:.4f}")
print("分类报告:")
print(report)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=final_model.classes_,
            yticklabels=final_model.classes_)
plt.xlabel("预测标签")
plt.ylabel("真实标签")
plt.title("HHO-BP 神经网络 - 故障分类混淆矩阵（优化后）")
plt.show()

# 深度学习部分的数据

In [2]:
!pip install pyts natsort -i https://pypi.tuna.tsinghua.edu.cn/simple some-package

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 6.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for some-package: filename=some_package-0.1-py3-none-any.whl size=1422 sha256=2eb5e880f29a1ed8732433f7ef100fb33331a7c54379c60f6af7d0b5b7e83686
  Stored in directory: /home/mw/.cache/pip/wheels/d7/e3/6a/7280c9d6d655157615c0306fb13fc466d4fe8b209e96f19722
Successfully built some-package


## 格拉姆角场转化

In [14]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from pyts.image import GramianAngularField
from pyts.approximation import PiecewiseAggregateApproximation

# === 1. 参数配置 ===
IMAGE_SIZE = 625  # 调整为与MATLAB生成的图像尺寸一致
SAMPLE_RANGE = (-1, 1)  # 归一化范围
COLORMAP = 'jet'  # MATLAB默认colormap
DPI = 100  # 分辨率控制
PAA_SIZE = 150    # PAA降维后的时间步数

# === 2. 加载数据 ===
data_2 = pd.read_csv('/home/mw/project/data_2.csv')
data = data_2.iloc[:, :-1].values  # 去掉最后一列

# === 3. 创建保存路径 ===
save_path_gadf = '格拉姆角场差'
os.makedirs(save_path_gadf, exist_ok=True)

# === 4. 初始化PAA和GAF转换器 ===
paa_transformer = PiecewiseAggregateApproximation(window_size=None, output_size=PAA_SIZE)

gadf_transformer = GramianAngularField(
    image_size=100,
    method='difference',
    sample_range=SAMPLE_RANGE,
    overlapping=False
)

# === 5. 生成并保存图像 ===
for i, sample in enumerate(data):
    # -- 应用PAA降维 --
    sample_paa = paa_transformer.transform(sample.reshape(1, -1))
    
    # -- 转换为GADF --
    gadf = gadf_transformer.fit_transform(sample_paa)[0]

    # -- GADF图像 --
    fig = plt.figure(figsize=(IMAGE_SIZE / DPI, IMAGE_SIZE / DPI), dpi=DPI)  # 将图像大小调整为6.25x6.25英寸，确保625像素
    plt.imshow(gadf, cmap=COLORMAP, origin='lower', aspect='auto')
    plt.axis('off')  # 去掉坐标轴
    plt.gca().set_position([0, 0, 1, 1])  # 去掉边框，填满整个画布
    plt.savefig(os.path.join(save_path_gadf, f"{i+1:04d}.jpg"),
                bbox_inches='tight',
                pad_inches=0,
                dpi=DPI)
    plt.close()

print("所有图像生成并保存完成！")


所有图像生成并保存完成！


In [3]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import glob
from natsort import natsorted
import warnings

def process_gaf_images(
    input_folder,           # 要处理的文件夹路径
    data_path,              # CSV数据文件路径
    output_path='output.npz',  # 输出文件完整路径
    target_size=(64, 64),   # 目标图像尺寸
    save_format='npz',      # 保存格式: 'npz' 或 'mat'
    verbose=True            # 是否打印进度信息
):
    # 参数验证
    assert save_format in ['npz', 'mat'], "仅支持 npz/mat 格式"
    
    # 创建输出目录（如果路径包含目录）
    output_dir = os.path.dirname(output_path)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # 加载标签数据
    try:
        labels = pd.read_csv(data_path).iloc[:, -1].values.flatten()
    except Exception as e:
        raise ValueError(f"加载CSV数据失败: {str(e)}")

    # 获取并排序图像文件
    image_files = natsorted(glob.glob(os.path.join(input_folder, '*.jpg')))
    if not image_files:
        warnings.warn(f"文件夹 {input_folder} 中没有找到jpg文件", UserWarning)
        return

    # 检查图像数量与标签数量是否一致
    assert len(image_files) == len(labels), (
        f"图像数量({len(image_files)})与标签数量({len(labels)})不匹配"
    )

    # 处理图像
    images = []
    for idx, img_path in enumerate(image_files):
        with Image.open(img_path) as img:
            if img.mode != 'RGB':
                img = img.convert('RGB')
            img_array = np.array(img.resize(target_size), dtype=np.float32) / 255.0
            images.append(img_array)
        
        if verbose and (idx + 1) % 100 == 0:
            print(f"已处理 {idx + 1}/{len(image_files)} 张图像")

    # 保存数据
    images, labels = np.array(images), np.array(labels)
    if save_format == 'npz':
        np.savez(output_path, images=images, labels=labels)
    else:
        from scipy.io import savemat
        savemat(output_path, {'resizeimg': images, 'labels': labels}, do_compression=True)

    if verbose:
        print(f"\n处理完成！保存文件: {output_path}")
        print(f"最终数据形状: 图像 {images.shape}, 标签 {labels.shape}")

if __name__ == "__main__":
    process_gaf_images(
        input_folder='/home/mw/project/格拉姆角场差',
        data_path='/home/mw/project/data_2.csv',
        output_path='格拉姆角场差.mat',
        target_size=(64, 64),
        save_format='mat',
        verbose=True
    )

已处理 100/1600 张图像
已处理 200/1600 张图像
已处理 300/1600 张图像
已处理 400/1600 张图像
已处理 500/1600 张图像
已处理 600/1600 张图像
已处理 700/1600 张图像
已处理 800/1600 张图像
已处理 900/1600 张图像
已处理 1000/1600 张图像
已处理 1100/1600 张图像
已处理 1200/1600 张图像
已处理 1300/1600 张图像
已处理 1400/1600 张图像
已处理 1500/1600 张图像
已处理 1600/1600 张图像

处理完成！保存文件: 格拉姆角场差.mat
最终数据形状: 图像 (1600, 64, 64, 3), 标签 (1600,)
